# Week 2
for this weeks assignment i chose to do text clustering. for this assignment i will pick out my own dataset and perform text clustering on it.


# Imports

In [1]:
import pandas as pd
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import re
import string
import glob
import pandas as pd
from pathlib import Path

In [14]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /homes/mkwierenga/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /homes/mkwierenga/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /homes/mkwierenga/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /homes/mkwierenga/nltk_data...


True

# Step 1: load the PDF file


In [2]:
reader = PyPDF2.PdfReader('data/machinelearningbook.pdf')

## step 2: extract the lines to dataframe

In [3]:
data = []
chapter = 0
for i in range(14, 369):
    page = reader.pages[i].extract_text().replace('\n', ' ').split('.')
    for line in page:
        if 'CHAPTER' in line:
            chapter +=1
        line = line.lower()
        line = re.sub('\[.*?\]', ' ', line)
        line = re.sub('[%s]' % re.escape(string.punctuation), ' ', line)
        line = re.sub('\w*\d\w*', ' ', line)
        line = re.sub('�', ' ', line)
        data.append([chapter, line, i])
df = pd.DataFrame(data, columns=['chapter', 'line', 'page_number'])

In [4]:
# Noun extract and lemmatize function
def nouns(text):
    '''Given a string of text, tokenize the text 
    and pull out only the nouns.'''
    # create mask to isolate words that are nouns
    is_noun = lambda pos: pos[:2] == 'NN'
    # store function to split string of words 
    # into a list of words (tokens)
    tokenized = word_tokenize(text)
    # store function to lemmatize each word
    wordnet_lemmatizer = WordNetLemmatizer()
    # use list comprehension to lemmatize all words 
    # and create a list of all nouns
    all_nouns = [wordnet_lemmatizer.lemmatize(word) \
    for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    
    #return string of joined list of nouns
    return ' '.join(all_nouns)



In [15]:
data_nouns = pd.DataFrame(df["line"].apply(nouns))
# Visually Inspect
data_nouns

,line
0,chapter introduction machine learning knowledg...
1,research field intersection statistic intellig...
2,application machine method year life
3,recommendation movie food order product online...
4,website facebook amazon netflix part site mach...


In [17]:
data_nouns

,line
0,chapter introduction machine learning knowledg...
1,research field intersection statistic intellig...
2,application machine method year life
3,recommendation movie food order product online...
4,website facebook amazon netflix part site mach...
...,...
7520,development text processing year scope book re...
7521,use vector representation word vector word rep...
7522,paper “ representation word phrase composi‐ ti...
7523,introduction subject


In [22]:


# Create a document-term matrix with only nouns
# Store TF-IDF Vectorizer
# tfidf_text_vectorizer = TfidfVectorizer(stop_words=list(STOP_WORDS))
tv_noun = TfidfVectorizer(stop_words=text.ENGLISH_STOP_WORDS, ngram_range = (1,1), max_df = .8, min_df = .01)
# Fit and Transform speech noun text to a TF-IDF Doc-Term Matrix
data_tv_noun = tv_noun.fit_transform(data_nouns.line)
# Create data-frame of Doc-Term Matrix with nouns as column names
data_dtm_noun = pd.DataFrame(data_tv_noun.toarray(), columns=tv_noun.get_feature_names_out())
data_dtm_noun.index = df.index
# Visually inspect Document Term Matrix
data_dtm_noun.head()



AttributeError: 'str' object has no attribute 'ENGLISH_STOP_WORDS'

# Step 2: clean the data